In [ ]:
#篩選features: 只要 bis hr sbp*3(不為NAN) propofol
#輸入:csv 輸出csv
import os
import pandas as pd
from pandas.core.frame import DataFrame
from pathlib import Path                #用來清除檔名
import json
from pandas import json_normalize   

pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)

path ="D:\\IL\\Result\\PASS" #檔案目錄(可以自行更改)
files = os.listdir(path) #得到文件所有名稱
#print(files)
position_list=[]
filename_list=[]    #寫出檔案時,檔名命名需要用到

for file in files:
    position = path +'\\'+ file
    position_list.append(position)
    file_name = Path(position).stem      #去檔名
    filename_list.append(file_name)
    
file_counter=0   #檔名命名用    
count=0
output=0

for i in position_list:
    s1=pd.DataFrame()
    s2=pd.DataFrame()
    s3=pd.DataFrame()
    s4=pd.DataFrame()
    s5=pd.DataFrame()
    print(count)
    count=count+1
    
    #每跑一次要清0
    S1=s1
    S2=s2
    S3=s3
    S4=s4
    S5=s5
    
    df1=pd.read_excel(i,sheet_name="Sheet_name_1",index_col=0)
    df2=pd.read_excel(i,sheet_name="Sheet_name_2",index_col=0)
    
    s3=df2["observedata_value_OPER"]
    S3=s3.fillna(1)
    
    if "machinedata_sbp" in df2.columns:
        check=True
        for j in range(0,len(df2.index)):
            if pd.isna(df2["machinedata_sbp"][j])== True:
                check = True
            else:
                check = False
                break
        
        if check == True:
            file_counter = file_counter + 1
            continue
        else:
            s1=df2[["machinedata_bis","machinedata_hr",
                    "machinedata_sbp","machinedata_dbp","machinedata_mbp"]]    
            S1=s1.interpolate()
    else:
        file_counter=file_counter+1
        continue
    
    if "bolusdrugorder_name_2% Propofol CE" in df2.columns:
        s4=df2["bolusdrugorder_value_2% Propofol CE"]
        S4=s4.fillna(method="ffill",axis=0)
   
    else:
        file_counter = file_counter + 1
        continue
    
    
    
    select=pd.concat([S1,S3,S4],axis=1,sort=False)
    select=select.fillna(0)
        

    
    out_path = "D:\\IL\\k_sbp"+"\\"+ filename_list[file_counter] +".xlsx"   #儲存資料路徑與檔案名稱可自行更改
    
    with pd.ExcelWriter(out_path) as writer:  
        df1.to_excel(writer, sheet_name='Sheet_name_1')
        select.to_excel(writer, sheet_name='Sheet_name_2')
    
    file_counter = file_counter + 1
    output=output+1
    print("output: ",output)